## Making a test widget...and generating the widget code

In [9]:
import tifffile as tff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import anndata as ad
import pickle as pk

from vitessce.data_utils import (
    multiplex_img_to_ome_zarr,
    optimize_adata,
    VAR_CHUNK_SIZE,
)
from vitessce import (
    VitessceConfig,
    VitessceChainableConfig, 
    VitessceConfigDatasetFile,
    Component as cm,
    CoordinationType as ct,
    AnnDataWrapper,
    OmeTiffWrapper,
    OmeZarrWrapper,
    CsvWrapper,
    MultiImageWrapper,
    AbstractWrapper,
)
from vitessce.export import export_to_files
import json

In [5]:
# source imports
import os
from os.path import join
import tempfile
from uuid import uuid4
from pathlib import PurePath, PurePosixPath
from vitessce.repr import make_repr

In [6]:
# Since this is such a generic class going to repurpose for JSON. All it really needs to do is make the routes...

class JSONWrapper(AbstractWrapper):

    """
    Wrap a JSON file by creating an instance of the ``JSONWrapper`` class.

    :param str data_type: The data type of the information contained in the file.
    :param str csv_path: A local filepath to a CSV file.
    :param str csv_url: A remote URL of a CSV file.
    :param dict options: The file options.
    :param dict coordination_values: The coordination values.
    :param \\*\\*kwargs: Keyword arguments inherited from :class:`~vitessce.wrappers.AbstractWrapper`
    """

    def __init__(self, json_path=None, json_url=None, data_type=None, options=None, coordination_values=None,
                 **kwargs):
        super().__init__(**kwargs)
        self._repr = make_repr(locals())
        self._json_path = json_path
        self._json_url = json_url
        self._data_type = data_type
        self._options = options
        self._coordination_values = coordination_values
        self.is_remote = json_url is not None
        self.local_json_uid = str(uuid4())
        if data_type is None:
            raise ValueError("Expected data_type to be provided")
        if json_url is not None and json_path is not None:
            raise ValueError(
                "Did not expect json_url to be provided with json_path")
        if json_url is None and json_path is None:
            raise ValueError(
                "Expected json_url or json_path to be provided")

    def convert_and_save(self, dataset_uid, obj_i, base_dir=None):
        # Only create out-directory if needed
        if not self.is_remote:
            super().convert_and_save(dataset_uid, obj_i, base_dir=base_dir)

        file_def_creator = self.make_json_file_def_creator(
            dataset_uid, obj_i)
        routes = self.make_json_routes(dataset_uid, obj_i)

        self.file_def_creators.append(file_def_creator)
        self.routes += routes

    def make_json_routes(self, dataset_uid, obj_i):
        if self.is_remote:
            return []
        else:
            # TODO: Move imports back to top when this is factored out.
            from vitessce.routes import FileRoute
            from starlette.responses import FileResponse

            if self.base_dir is not None:
                local_json_path = join(self.base_dir, self._json_path)
                local_json_route_path = file_path_to_url_path(self._json_path)
            else:
                local_json_path = self._json_path
                local_json_route_path = self._get_route_str(dataset_uid, obj_i, self.local_json_uid)

            async def response_func(req):
                return FileResponse(local_json_path, filename=os.path.basename(self._json_path))
            routes = [
                FileRoute(local_json_route_path, response_func, local_json_path),
            ]
            return routes

    def make_json_file_def_creator(self, dataset_uid, obj_i):
        def json_file_def_creator(base_url):
            file_def = {
                "fileType": f"{self._data_type}.json",
                "url": self.get_json_url(base_url, dataset_uid, obj_i),
            }
            if self._options is not None:
                file_def["options"] = self._options
            if self._coordination_values is not None:
                file_def["coordinationValues"] = self._coordination_values
            return file_def
        return json_file_def_creator

    def get_json_url(self, base_url="", dataset_uid="", obj_i=""):
        if self.is_remote:
            return self._json_url
        if self.base_dir is not None:
            return self._get_url_simple(base_url, file_path_to_url_path(self._json_path, prepend_slash=False))
        return self._get_url(base_url, dataset_uid,
                             obj_i, self.local_json_uid)

In [4]:
adata = ad.read_h5ad("../data/test/adata_small.h5ad")

In [2]:
zarr_filepath = "../data/test/converted/adata_small.zarr"

## Making the widget

In [ ]:
# Option with ome-zarr
vc = VitessceConfig(schema_version="1.0.15", name='CODEX', description='Testing small slice of CODEX image')
dataset = vc.add_dataset(name='R1reg1').add_object(
    OmeZarrWrapper(img_path="/mnt/isilon/cscb/codex/pillaiv/SCTC-VP-15/vitessce-gh-pages/data/test/converted/codex_img.ome.zarr")
)
spatial = vc.add_view(cm.SPATIAL, dataset=dataset)
status = vc.add_view(cm.STATUS, dataset=dataset)
lc = vc.add_view(cm.LAYER_CONTROLLER, dataset=dataset).set_props(disableChannelsIfRgbDetected=True)
vc.layout(spatial | lc / status);

In [7]:
# Option with ome.tiff
vc = VitessceConfig(schema_version="1.0.15", name='CODEX', description='Testing small slice of CODEX image')
# dataset = vc.add_dataset(name='R1reg1').add_object(
#     MultiImageWrapper(
#         image_wrappers=[
#             OmeTiffWrapper(img_path='/mnt/isilon/cscb/codex/pillaiv/SCTC-VP-15/vitessce-gh-pages/data/test/raw/codex.ome.tif', name='Image'),
#             OmeTiffWrapper(img_path='/mnt/isilon/cscb/codex/pillaiv/SCTC-VP-15/vitessce-gh-pages/data/test/raw/masks.ome.tif', name='Mask', is_bitmask=True),
#         ]
#  )
# )

# dataset = vc.add_dataset(name='R1reg1').add_object(
#     OmeTiffWrapper(img_path='/mnt/isilon/cscb/codex/pillaiv/SCTC-VP-15/vitessce-gh-pages/data/test/raw/codex.ome.tif', name='Image')
#  )

dataset = vc.add_dataset(name='R1reg1_obs').add_object(AnnDataWrapper(
    adata_path=zarr_filepath,
    obs_set_paths=["obs/new_pheno"],
    obs_set_names=["phenotype"],
    obs_embedding_paths=["obsm/X_umap", "obsm/X_pca"],
    obs_embedding_names=["UMAP", "PCA"],
    obs_feature_matrix_path="X"
))

# dataset = dataset.add_object(AnnDataWrapper(
#     adata_path=zarr_filepath,
#     obs_set_paths=["obs/new_pheno"],
#     obs_set_names=["phenotype"],
#     obs_embedding_paths=["obsm/X_umap", "obsm/X_pca"],
#     obs_embedding_names=["UMAP", "PCA"],
#     obs_feature_matrix_path="X"
# ))

dataset = dataset.add_object(JSONWrapper(json_path="/mnt/isilon/cscb/codex/pillaiv/SCTC-VP-15/vitessce-gh-pages/data/test/converted/test_segmentations.json",
                                        data_type="obsSegmentations"))

spatial = vc.add_view(cm.SPATIAL, dataset=dataset)
# status = vc.add_view(cm.STATUS, dataset=dataset)
lc = vc.add_view(cm.LAYER_CONTROLLER, dataset=dataset)
umap = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="UMAP")
cell_sets = vc.add_view(cm.OBS_SETS, dataset=dataset)
genes = vc.add_view(cm.FEATURE_LIST, dataset=dataset)
vc.layout(spatial | (lc | umap));

In [8]:
# exporting to file....
# config_dict = vc.export(to='files', base_url= 'http://localhost:3000', out_dir='../data/test/converted/webexport')
# config_dict

# export to core website??
config_dict = vc.export(to='files', base_url= 'https://cscb.research.chop.edu/', out_dir='../data/test/converted/webexport')
config_dict

{'version': '1.0.15',
 'name': 'CODEX',
 'description': 'Testing small slice of CODEX image',
 'datasets': [{'uid': 'A',
   'name': 'R1reg1_obs',
   'files': [{'fileType': 'anndata.zarr',
     'url': 'http://localhost:3000/A/0/5e3a83c0-4f0c-4263-84ba-fa2dd3e10687',
     'options': {'obsEmbedding': [{'path': 'obsm/X_umap',
        'dims': [0, 1],
        'embeddingType': 'UMAP'},
       {'path': 'obsm/X_pca', 'dims': [0, 1], 'embeddingType': 'PCA'}],
      'obsSets': [{'name': 'phenotype', 'path': 'obs/new_pheno'}],
      'obsFeatureMatrix': {'path': 'X'}}},
    {'fileType': 'obsSegmentations.json',
     'url': 'http://localhost:3000/A/1/8dd7b44f-e4b6-47f7-a05e-b0659b1401b5'}]}],
 'coordinationSpace': {'dataset': {'A': 'A'}, 'embeddingType': {'A': 'UMAP'}},
 'layout': [{'component': 'spatial',
   'coordinationScopes': {'dataset': 'A'},
   'x': 0.0,
   'y': 0,
   'w': 6.0,
   'h': 12},
  {'component': 'layerController',
   'coordinationScopes': {'dataset': 'A'},
   'x': 6.0,
   'y': 0,
 

In [7]:
vc.display()

In [33]:
vw = vc.widget()
print(vw)

VitessceWidget(config={'version': '1.0.15', 'name': 'CODEX', 'description': 'Testing small slice of CODEX image', 'datasets': [{'uid': 'A', 'name': 'R1reg1', 'files': [{'fileType': 'raster.json', 'options': {'schemaVersion': '0.0.2', 'usePhysicalSizeScaling': False, 'images': [{'name': 'Image', 'type': 'ome-tiff', 'url': 'http://localhost:8006/A/0/b80e0e0f-8df9-4a4b-a755-0b1a7352c1a7', 'metadata': {'omeTiffOffsetsUrl': 'http://localhost:8006/A/0/6d8e0b09-cce8-4e73-86b2-6a8d49fd1285', 'isBitmask': False}}, {'name': 'Mask', 'type': 'ome-tiff', 'url': 'http://localhost:8006/A/0/32ecd70a-b3e1-4291-b90a-95099b87af5f', 'metadata': {'omeTiffOffsetsUrl': 'http://localhost:8006/A/0/3cf58c94-232a-41d1-8370-c850d194b1cd', 'isBitmask': True}}], 'renderLayers': ['Image', 'Mask']}}, {'fileType': 'anndata.zarr', 'url': 'http://localhost:8006/A/1/0d4dd533-765f-4ac6-bf27-10e4434cbfef', 'options': {'obsEmbedding': [{'path': 'obsm/X_umap', 'dims': [0, 1], 'embeddingType': 'UMAP'}, {'path': 'obsm/X_pca', 

## Widget from HubMap

In [2]:
conf = VitessceChainableConfig(
    name="R1_reg1_expr", description="", schema_version="1.0.7"
).add_dataset(
    uid="A",
    name="R1_reg1",
    files=[
        VitessceConfigDatasetFile(
            # data_type="cells",
            file_type="anndata-cells.zarr",
            url="http://localhost:8000/B/0/e40a1d90-2dd4-4dce-a886-950bc9f22ffb",
            options={
                "xy": "obsm/centroids",
                "mappings": {"UMAP": {"key": "obsm/X_umap", "dims": [0, 1]}},
                "factors": [
                    "obs/new_pheno",
                    "obs/motif",
                ],
            },
        ),
        VitessceConfigDatasetFile(
            # data_type="cell-sets",
            file_type="anndata-cell-sets.zarr",
            url="http://localhost:8000/B/0/e40a1d90-2dd4-4dce-a886-950bc9f22ffb",
            options=[
                {
                    "groupName": "cell type",
                    "setName": "obs/new_pheno",
                },
                {
                    "groupName": "motif",
                    "setName": "obs/motif",
                },
            ],
        ),
        VitessceConfigDatasetFile(
            # data_type="expression-matrix",
            file_type="anndata-expression-matrix.zarr",
            url="http://localhost:8000/B/0/e40a1d90-2dd4-4dce-a886-950bc9f22ffb",
            options={"matrix": "X"},
        ),
        VitessceConfigDatasetFile(
            # data_type="raster",
            file_type="raster.json",
            options={
                "schemaVersion": "0.0.2",
                "usePhysicalSizeScaling": False,
                "images": [
                    {
                        "name": "reg001_expr",
                        "type": "ome-tiff",
                        "url": "http://localhost:8000/A/0/8dd18301-c8d5-448c-a6ae-9f1b3674672e",
                        "metadata": {
                            "omeTiffOffsetsUrl": "http://localhost:8000/A/0/2c362cab-8604-40aa-9aa6-6a8b0c72aac2",
                            "isBitmask": False,
                        },
                    },
                    {
                        "name": "reg001_mask",
                        "type": "ome-tiff",
                        "url": "http://localhost:8000/A/0/d501447b-100c-4933-a26b-7abb33884e4e",
                        "metadata": {
                            "omeTiffOffsetsUrl": "http://localhost:8000/A/0/fa36ae30-c4eb-43d9-918d-043e43238556",
                            "isBitmask": True,
                        },
                    },
                ],
                "renderLayers": ["reg001_expr", "reg001_mask"],
            },
        ),
    ],
).set_coordination_value(
    c_type="embeddingType", c_scope="A", c_value="UMAP"
).add_view(
    dataset_uid="A", component="description", x=0, y=8, w=3, h=4
).add_view(
    dataset_uid="A", component="layerController", x=0, y=0, w=3, h=8
).add_view(
    dataset_uid="A", component="spatial", x=3, y=0, w=4, h=8
).add_view(
    dataset_uid="A",
    component="scatterplot",
    x=7,
    y=0,
    w=3,
    h=8,
    coordination_scopes={"embeddingType": "A"},
).add_view(
    dataset_uid="A", component="cellSets", x=10, y=5, w=2, h=7
).add_view(
    dataset_uid="A",
    component="genes",
    x=10,
    y=0,
    w=2,
    h=5,
    props={"variablesLabelOverride": "antigen"},
).add_view(
    dataset_uid="A",
    component="heatmap",
    x=3,
    y=8,
    w=7,
    h=4,
    props={"variablesLabelOverride": "antigen", "transpose": True},
)

conf.widget()

VitessceWidget(config={'version': '1.0.7', 'name': 'R1_reg1_expr', 'description': '', 'datasets': [{'uid': 'A'…

In [12]:
# print(conf.widget())
conf_dict = export_to_files(conf, base_url="http://localhost")
with open("../src/schema1.0.7_template.json","w") as f:
    json.dump(conf_dict, f)

In [14]:
conf_dict

{'version': '1.0.7',
 'name': 'R1_reg1_expr',
 'description': '',
 'datasets': [{'uid': 'A',
   'name': 'R1_reg1',
   'files': [{'fileType': 'anndata-cells.zarr',
     'url': 'http://localhost:8000/B/0/e40a1d90-2dd4-4dce-a886-950bc9f22ffb',
     'options': {'xy': 'obsm/centroids',
      'mappings': {'UMAP': {'key': 'obsm/X_umap', 'dims': [0, 1]}},
      'factors': ['obs/new_pheno', 'obs/motif']}},
    {'fileType': 'anndata-cell-sets.zarr',
     'url': 'http://localhost:8000/B/0/e40a1d90-2dd4-4dce-a886-950bc9f22ffb',
     'options': [{'groupName': 'cell type', 'setName': 'obs/new_pheno'},
      {'groupName': 'motif', 'setName': 'obs/motif'}]},
    {'fileType': 'anndata-expression-matrix.zarr',
     'url': 'http://localhost:8000/B/0/e40a1d90-2dd4-4dce-a886-950bc9f22ffb',
     'options': {'matrix': 'X'}},
    {'fileType': 'raster.json',
     'options': {'schemaVersion': '0.0.2',
      'usePhysicalSizeScaling': False,
      'images': [{'name': 'reg001_expr',
        'type': 'ome-tiff',
   

In [20]:
conf.display()